In [1]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
import numpy as np
import pandas as pd
import os
import re
import glob
import tqdm
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg
import cv2
import geopandas as gp
import descartes
from shapely.geometry import Polygon
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import KNNImputer

from SolCrawler import solpos

In [5]:
!pip install descartes

  Using cached https://files.pythonhosted.org/packages/e5/b6/1ed2eb03989ae574584664985367ba70cd9cf8b32ee8cad0e8aaeac819f3/descartes-1.1.0-py3-none-any.whl


## load grid data and weather station data

In [3]:
# load grid data
taiwan_grid = gp.read_file("./MapData/taiwan_grid.shp")
taiwan_offgrid = gp.read_file("./MapData/taiwan_offgrid.shp")

assert taiwan_grid.crs == 'epsg:3824' 
assert taiwan_offgrid.crs == 'epsg:3824' 


# get centroids of each grid
# make centrod as each grid's 'longitude' 'latitude'
centroid = taiwan_grid['geometry'].centroid
taiwan_grid['longitude'] = centroid.x
taiwan_grid['latitude'] = centroid.y
#taiwan_grid


# load the weather station info including 'longitude' 'latitude'
Sta_df = pd.read_csv('WeatherStation.csv')
# make Geodataframe (geometry points) out of the 'longitude' 'latitude'
## to locate all the stations in the grid
Sta_gdf = gp.GeoDataFrame(
    Sta_df, geometry=gp.points_from_xy(Sta_df.Longitude, Sta_df.Latitude), crs=taiwan_grid.crs)

## load fill_CODiS data

In [7]:
# loop over the fill_CODiS folder
fill_csv = sorted(glob.glob('fill_CODiS/*.csv'))
#print(fill_csv)


# ['StnPres', 'Temperature'] min and max
## to get the original 'StnPres', 'Temperature' for the SOLPOS input 
p_max, t_max = pd.read_csv("./scale_max.csv").iloc[:2, 0]  
p_min, t_min = pd.read_csv("./scale_min.csv").iloc[:2, 0]
#print(p_max, t_max, p_min, t_min)

# for split the alltime into time-by-time dataframe
## split by the total number of grids (66)
split = taiwan_grid.shape[0]

NameError: name 'taiwan_grid' is not defined

## locate each station, and count the number of stations in each cell of the grid

In [ ]:
fields = [[] for i in range(len(taiwan_grid))]
for i in tqdm.tqdm_notebook(range(len(Sta_gdf))):
    for u in range(len(taiwan_grid)):
        # check if the geometry point of station is in the geometry polygon of grid
        if Sta_gdf.loc[i, 'geometry'].within(taiwan_grid.loc[u, 'geometry']):
            fields[u].append(i)
#fields

## KNN impute the whole grid, and query for the SOLPOS

In [ ]:
# for searching the digital number in the file name
## 2017-01-01_H01
p = re.compile(r'\d+')

# loop over the fill_CODiS folder
for name in tqdm.tqdm_notebook(fill_csv[475:]):
    #print(name)

    # load each fill_CODiS time by time
    codis = pd.read_csv(name)

    # initiate features in the taiwan_grid 
    for i in ['StnPres', 'Temperature', 'RH', 'WS', 'WD_sin', 'WD_cos', 'Precp', 'AM', 'CosInc', 'Hour', 'ETR']:
        taiwan_grid[i] = 0.0
    #taiwan_grid

    # loop over the count of stations in each cell
    for u, i in enumerate(fields):
        if len(i)>0:
            for j in i:
                ## if the cell has stations, then accumulate the values in the fill_CODiS
                # 4:11 & 7:14 ['StnPres', 'Temperature', 'RH', 'WS', 'WD_sin', 'WD_cos', 'Precp']
                taiwan_grid.iloc[u, 4:11] += codis.iloc[j, 7:14]
            # take average value for each cell
            taiwan_grid.iloc[u, 4:11] /= len(i)
        else:
            # if the cell has no station located, then set as nan
            taiwan_grid.iloc[u, 4:11] =np.nan
    
    # impute the rest of the cell (the nan) by KNN time by time, make the whole grid filled 
    ## the KNN takes scaled data (0.0-1.0)
    imputer = KNNImputer(n_neighbors=3)#, copy=True)
    taiwan_grid[['longitude', 'latitude', 'StnPres', 'Temperature', 'RH', 'WS', 'WD_sin', 'WD_cos', 'Precp']] = \
    imputer.fit_transform(taiwan_grid[['longitude', 'latitude', 'StnPres', 'Temperature', 'RH', 'WS', 'WD_sin', 'WD_cos', 'Precp']])

    # reverse the scale to original "StnPres", "Temperature" for the SOLPOS
    taiwan_grid['StnPres_ori'] = taiwan_grid['StnPres']*(p_max-p_min)+p_min
    taiwan_grid['Temperature_ori'] = taiwan_grid['Temperature']*(t_max-t_min)+t_min
    #print(taiwan_grid)

    #query for SOLPOS
    date = p.findall(name) # digital number in the file name
    #print(date)

    # per day
    syear, smonth, sday, hour = date
    eyear, emonth, eday = syear, smonth, sday
    # for a certain time (hour) in a day
    hour = int(hour)

    # the query is each cell's 'latitude', 'longitude', 'StnPres_ori', 'Temperature_ori'
    ## hence, the whole grid could be filled
    for i in range(len(taiwan_grid)):
        StaLoc = dict(taiwan_grid.loc[i, ['latitude', 'longitude', 'StnPres_ori', 'Temperature_ori']])
        #print(StaLoc)
        
        # call the custom function
        am, cosinc, etr = solpos(syear=syear,
                         smonth=smonth,
                         sday=sday,
                         eyear=eyear,
                         emonth=emonth,
                         eday=eday,
                         hour=hour,
                         **StaLoc)

        # initiate hour as 1.0, meaning sunlight is on
        ## this is the sunshine hour in the space
        hr = 1.0

        # if the sunlight is off, then no AirMass, no incidence, and no sunshine hour
        if etr == 0.0:
            am, cosinc, hr = 0.0, 0.0, 0.0

        # fill the grid time by time (hour by hour)
        taiwan_grid.loc[i, ['AM', 'CosInc', 'Hour', 'ETR']] = (am, cosinc, hr, etr)
        #print(am, cosinc, hr, etr)
        
    # save as temporary archive hour by hour
    taiwan_grid.iloc[:, 2:].to_csv('grid_'+name, index=False)

#     alltime_grid = pd.concat((alltime_grid, taiwan_grid))

## Concate the grid for the all-time grid to scale 'AM' 'CosInc'

In [29]:
# concate the temporary archive
## for future use, no need for the "pd.read_csv('alltime_grid.csv')"
alltime_grid = pd.DataFrame(columns = cols)
#alltime_grid = pd.read_csv('alltime_grid.csv')
grid_csv = sorted(glob.glob('grid_fill_CODiS/grid_Station_bytime_2017/*.csv'))
for i in grid_csv:
    grid_tmp = pd.read_csv(i)
    alltime_grid = pd.concat((alltime_grid, grid_tmp))
# for future use, no ignore_index "alltime_grid = pd.concat((alltime_grid, grid_tmp))"
## cuz it is usefule for the later maych-up between two dataframe with same index 

In [11]:
# reset the index  470/744 [7:19:09<4:35:44, 57.45s/it]
## should comment it for future use
alltime_grid.index = range(len(alltime_grid))


# scale the 'AM' 'CosInc' across all time
sol_scaler = MinMaxScaler((0, 1), copy=True)
# -4: [AM, CosInc, ETR]
## except 'Hour', it is defined in the function as 0.0/1.0
alltime_grid.loc[:, ['AM', 'CosInc']] = sol_scaler.fit_transform(alltime_grid.loc[:, ['AM', 'CosInc']])

#assert alltime_grid.crs == 'epsg:3824'

alltime_grid.describe()

KeyError: "None of [Index(['AM', 'CosInc'], dtype='object')] are in the [columns]"

In [58]:
cols = pd.read_csv('./fill_CODiS/fill2_Station_bytime_2017/2017-01-20_H23.csv').columns
cols     #et091-group6/fill_CODiS/fill2_Station_bytime_2017/2017-01-01_H08.csv

Index(['Stations', 'Altitude', 'Latitude', 'Longitude', 'altitude_s',
       'latitude_s', 'longitude_s', 'StnPres', 'Temperature', 'RH', 'WS',
       'WD_sin', 'WD_cos', 'Precp', 'SunShine', 'GloblRad'],
      dtype='object')

In [64]:
alltime_grid = pd.DataFrame(columns = cols)
#alltime_grid = pd.read_csv('alltime_grid.csv')
grid_csv = sorted(glob.glob('fill_CODiS/fill2_Station_bytime_2017/*.csv'))
for i in grid_csv:
    grid_tmp = pd.read_csv(i)
    alltime_grid = pd.concat((alltime_grid, grid_tmp[:9]))

In [65]:
alltime_grid.describe()

,Altitude,Latitude,Longitude,altitude_s,latitude_s,longitude_s,StnPres,Temperature,RH,WS,WD_sin,WD_cos,Precp,SunShine,GloblRad
count,78840.000000,78840.000000,78840.000000,78840.000000,78840.000000,78840.000000,78840.000000,78840.000000,78840.000000,78840.000000,78840.000000,78840.000000,78840.000000,78840.000000,78840.000000
mean,284.933333,22.867889,120.686822,0.117332,0.567774,0.452985,0.948007,0.610812,0.753698,0.131543,0.501320,0.718085,0.001301,0.231342,0.615728
std,752.632896,0.473199,0.393556,0.312114,0.277813,0.296866,0.074641,0.143222,0.112032,0.093615,0.302842,0.332178,0.010509,0.382110,0.943352
min,2.300000,22.003900,120.204800,0.000124,0.060530,0.089387,0.723449,0.030374,0.130000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8.100000,22.566000,120.315700,0.002530,0.390536,0.173041,0.969133,0.521028,0.680000,0.059783,0.250000,0.500000,0.000000,0.000000,0.000000
50%,22.300000,22.993200,120.746300,0.008418,0.641343,0.497850,0.973088,0.640187,0.750000,0.108696,0.500000,0.883022,0.000000,0.000000,0.000000
75%,33.500000,23.097500,120.903800,0.013063,0.702577,0.616655,0.977428,0.717290,0.830000,0.179348,0.750000,0.969846,0.000000,0.400000,1.030000
max,2413.400000,23.508200,121.373400,1.000000,0.943698,0.970883,1.000000,0.932243,1.000000,0.804348,1.000000,1.000000,0.410468,1.000000,4.360000


In [63]:
sum(alltime_grid.isna()['GloblRad'])

0

## drop the some useless features, keep the useful features for generating featmaps

In [7]:
alltime_grid = alltime_grid.drop(['longitude', 'latitude', 'StnPres', 'Temperature', 'StnPres_ori', 'Temperature_ori'], axis=1)

## Generate featmaps

In [1]:
# initiate the taiwan_grid and taiwan_offgrid
taiwan_grid = gp.read_file("./MapData/taiwan_grid.shp")
taiwan_offgrid = gp.read_file("./MapData/taiwan_offgrid.shp")

for i in ['RH', 'WS', 'WD_sin', 'WD_cos', 'Precp', 'AM', 'CosInc', 'Hour', 'ETR']:
    taiwan_grid[i] = 0.0
    taiwan_offgrid[i] = 0.0 # taiwan_offgrid values will be always 0.0

# params for the image plot
## trial&error for the pixel values
### to match the 200x155 image size
dpi = 100
width_pixel = 200
height_pixel = 350

# the scope is bounds of taiwan_offgrid 
left_bound = taiwan_offgrid.total_bounds[0]
right_bound = taiwan_offgrid.total_bounds[2]
bottom_bound = taiwan_offgrid.total_bounds[1]
top_bound = taiwan_offgrid.total_bounds[3]

# params for slicing the image array (clip the image)
left = 25
right = 179
top = 74
bottom = 273

# loop over the fill_CODiS hour by hour
for i in tqdm.tqdm_notebook(range(len(fill_csv))):
#for i in tqdm.tqdm_notebook(range(3)):
    
    # 2017-01-01_H01
    npy_name = re.search(r'\d{4}-\d{2}-\d{2}_H\d{2}', fill_csv[i]).group()
    
    # split the all-time grid into each time slot (per hour)
    ## match two dataframe by the same index range
    taiwan_grid.loc[:, ['RH', 'WS', 'WD_sin', 'WD_cos', 'Precp', 'AM', 'CosInc', 'Hour', 'ETR']] = \
    alltime_grid.loc[(i*split):((i+1)*split-1), ['RH', 'WS', 'WD_sin', 'WD_cos', 'Precp', 'AM', 'CosInc', 'Hour', 'ETR']].reset_index(drop=True)
    
    # concate taiwan_grid with taiwan_offgrid to plot a whole grid
    featmap = pd.concat([taiwan_grid, taiwan_offgrid], ignore_index=True)
    #print(featmap)
    
        # generate featmaps by using plt.plot to acquire image array
        ## the plt.plot will scale the range from 0.0-1.0 up to 0-255
    for feat in ['RH', 'WS', 'WD_sin', 'WD_cos', 'Precp', 'AM', 'CosInc']:
    
        # set the figsize by inches
        fig, ax = plt.subplots(figsize=(width_pixel/dpi, height_pixel/dpi), dpi=dpi)
        # acquire the the canvas for the figure
        canvas = FigureCanvasAgg(fig)
        
        # add the max range to make sure the scaled scope of each img (time after time) is same
        ## since all the value is scaled between 0.0 to 1.0, then the max is 1.0
        featmap.plot(ax=ax, column=feat, cmap='gray', vmax=1.0)
        #print(featmap)

        ax.set_axis_off()
        ax.set_xlim(left=left_bound, right=right_bound)
        ax.set_ylim(bottom=bottom_bound, top=top_bound)

        # Retrieve a view on the renderer buffer
        canvas.draw()
        buf = canvas.buffer_rgba()
        X = np.asarray(buf)

# comment after knowing the values
#         assert sum(sum(X[:,:,0] != X[:,:,1])) == 0
#         assert sum(sum(X[:,:,2] != X[:,:,1])) == 0
        
        # make sure the three channels are equal
        ## output should be one channel for each feature
        X = X[:,:,0]

# comment after knowing the values
#         width_mean = X.mean(axis=0)
#         height_mean = X.mean(axis=1)
#         width_ind = np.arange(X.shape[-1])
#         height_ind = np.arange(X.shape[0])
#         left = width_ind[list(width_mean != 255.0)][0]
#         right = width_ind[list(width_mean != 255.0)][-1]
#         top = height_ind[list(height_mean != 255.0)][0]
#         bottom = height_ind[list(height_mean != 255.0)][-1]
#         print(left, right, top, bottom)

        img = X[top:bottom+1, left:right+1]
        #print(img)
        
        # add the max range to make sure the scale scope of each img
#         img[scale_mask] = 0.0
        #print(img)
        
        # save as numpy array
        np.save(f'./FeatMap/{feat}/{npy_name}.npy', img)
        #cv2.imwrite(f'./FeatMap/{feat}/{img_name}.png', img)
        #plt.imshow(img)
        plt.close()
        
        
        # generate featmaps by filling the original values into corresponding array
        ## applying the boolean mask for each cell 
        for feat in ['Hour', 'ETR']:
            predmap = np.zeros((200, 155), dtype='float32')
        
            for grid in range(split):
                #print(sta,grid)
                predmap[np.load(f'./Grid_BoolMask/grid_{grid}.npy')] = featmap.loc[grid, feat]
            
            # save as numpy array    
            np.save(f'./FeatMap/{feat}/{npy_name}.npy', predmap)

NameError: name 'gp' is not defined

In [10]:
for feat in ['RH', 'WS', 'WD_sin', 'WD_cos', 'Precp', 'AM', 'CosInc', 'Hour', 'ETR']:
    assert len(os.listdir(f"./FeatMap/{feat}")) == (len(fill_csv) +1) # 1 for .ipynb checkpoint